## Proyecto final
### MongoDB: Pedidos y Ordenes

In [81]:
import pandas as pd
import numpy as np
from datetime import datetime
#import pprint
#from bson.objectid import ObjectId
import pymongo
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

In [9]:
conn_str = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn_str,
                             server_api=ServerApi('1'),
                             serverSelectionTimeoutMS=5000)

# Probando conexión
try:
    print(client.server_info())
except Exception:
    print("Unable to connect to the server.")

# Borrar base y volver a crear
# client.drop_database('BDBicis')

db = client['Tienda']

{'version': '5.0.8', 'gitVersion': 'c87e1c23421bf79614baf500fda6622bd90f674e', 'modules': [], 'allocator': 'tcmalloc', 'javascriptEngine': 'mozjs', 'sysInfo': 'deprecated', 'versionArray': [5, 0, 8, 0], 'openssl': {'running': 'OpenSSL 1.1.1f  31 Mar 2020', 'compiled': 'OpenSSL 1.1.1f  31 Mar 2020'}, 'buildEnvironment': {'distmod': 'ubuntu2004', 'distarch': 'aarch64', 'cc': '/opt/mongodbtoolchain/v3/bin/gcc: gcc (GCC) 8.5.0', 'ccflags': '-Werror -include mongo/platform/basic.h -fasynchronous-unwind-tables -ggdb -Wall -Wsign-compare -Wno-unknown-pragmas -Winvalid-pch -fno-omit-frame-pointer -fno-strict-aliasing -O2 -march=armv8.2-a -mtune=generic -Wno-unused-local-typedefs -Wno-unused-function -Wno-deprecated-declarations -Wno-unused-const-variable -Wno-unused-but-set-variable -Wno-missing-braces -fstack-protector-strong -Wa,--nocompress-debug-sections -moutline-atomics', 'cxx': '/opt/mongodbtoolchain/v3/bin/g++: g++ (GCC) 8.5.0', 'cxxflags': '-Woverloaded-virtual -Wno-maybe-uninitialize

### Colecciones:

* Pedidos
* Órdenes

In [83]:
# Registro de una nueva orden
def nueva_orden(compra:list, fecha:str = datetime.today().strftime('%Y-%m-%d')):
    total_orden = sum([x[2] for x in compra])
    compra = [{'producto_id': x[0], 'cantidad': x[1], 'subtotal': x[2]} for x in compra]
    orden = {'fecha': fecha ,
             'compra': compra,
             'total_orden': total_orden }
    
    orden_id = db.Ordenes.insert_one(orden).inserted_id

    return orden_id

# Registro de un nuevo pedido
def nuevo_pedido(fecha_entrega:str, nombre_cliente:str, telefono:str, compra:list, fecha_apertura:str = datetime.today().strftime('%Y-%m-%d')):
    total_orden = sum([x[2] for x in compra])
    compra = [{'producto_id': x[0], 'cantidad': x[1], 'subtotal': x[2]} for x in compra] 
    pedido = {'fecha_apertura': fecha_apertura ,
             'fecha_entrega': fecha_entrega,
             'nombre_cliente': nombre_cliente,
             'telefono': telefono,
             'compra': compra,
             'total_orden': total_orden }
    
    pedido_id = db.Pedidos.insert_one(pedido).inserted_id

    return pedido_id

## Queries:

1. Total de venta de ordenes por día.
4. Consultar pedidos por fecha de entrega.
2. Total de dinero generado por día por anticipo de pedidos.
2. Total de dinero generado por pago de restante de pedidos por día.
5. Consultar total de cada producto a entregar por día

In [89]:
# 1.Total de venta de órdenes por día:
def venta_ordenes_por_dia(fecha:str):
    #fecha = fecha.strftime('%Y-%m-%d')
    total = db.Ordenes.aggregate([{ "$match": { "fecha": fecha}},
                                  { "$group": { "_id": "$fecha", "total_anticipos_dia" : {'$sum': '$total_orden'} }},
                                   { "$project": { "_id":0, "total_anticipos_dia" : 1 }}])
    res = []                     
    for t in total:
        res.append(t)
    return res[0]

# 2. Consultar pedidos por fecha de entrega
def consultar_entregas(fecha_entrega:str = datetime.today().strftime('%Y-%m-%d')):
    #fecha = fecha.strftime('%Y-%m-%d')
    resultado = db.Pedidos.find({ "fecha_entrega": fecha_entrega}).sort('fecha_apertura')

    pedidos = []                     
    for p in resultado:
        pedidos.append(p)
    return pedidos


# 3. Total de pagos de anticipos por día
def pago_anticipos_por_dia(fecha_apertura:str = datetime.today().strftime('%Y-%m-%d')):
    #fecha = fecha.strftime('%Y-%m-%d')
    total = db.Pedidos.aggregate([{ "$match": { "fecha_apertura": fecha_apertura}},
                                   { "$group": { "_id": "$fecha_apertura", "total_anticipos_dia" : {'$sum': '$total_orden'} }},
                                   { "$project": { "_id":0, "total_anticipos_dia" : 1 }}])
    res = []                     
    for t in total:
        res.append(t)
    return res[0]['total_anticipos_dia']*0.5


# 4. Total de pagos de entregas por día
def pago_entregas_por_dia(fecha_entrega:str = datetime.today().strftime('%Y-%m-%d')):
    #fecha = fecha.strftime('%Y-%m-%d')
    total = db.Pedidos.aggregate([{ "$match": { "fecha_entrega": fecha_entrega}},
                                   { "$group": { "_id": "fecha_entrega", "total_pago_entregas" : {'$sum': '$total_orden'} }},
                                   { "$project": { "_id":0, "total_pago_entregas" : 1 }}])
    res = []                     
    for t in total:
        res.append(t)
    return res[0]['total_pago_entregas']*0.5


#5. Consultar cantidades de productos a entregar por día
def productos_para_entregas(fecha_entrega:str = datetime.today().strftime('%Y-%m-%d')):
    resultado = db.Pedidos.aggregate([{ "$match": { "fecha_entrega": fecha_entrega}},
                                      { "$unwind": "$compra" },
                                      { "$group": { "_id": "$compra.producto_id", "cantidad_total" : {'$sum':'$compra.cantidad'} }},
                                      { "$project": { "_id":1, "cantidad_total" : 1 }}])
    pedidos = []                     
    for p in resultado:
        pedidos.append(p)
    return pedidos


### Testing

In [60]:
# Crear ordenes:
compra_1 = [['jamon', 3, 45], ['piña', 1, 15], ['zarzamora', 2, 40]]

nueva_orden(compra_1)

ObjectId('629aa202abb83a99715e3bd3')

In [66]:
hoy = datetime.today().strftime('%Y-%m-%d')

print(venta_ordenes_por_dia(hoy))

{'total_anticipos_dia': 300}

In [69]:
# Crear un pedido:
compra_2 = [['jamon', 6, 90], ['piña', 2, 30], ['zarzamora', 4, 80]]

nuevo_pedido('2022-06-10','Marcela Cruz', '93838381005', compra_2)

ObjectId('629aa29aabb83a99715e3bd4')

In [92]:
consultar_entregas('2022-06-10')

[{'_id': ObjectId('629aa29aabb83a99715e3bd4'),
  'fecha_apertura': '2022-06-03',
  'fecha_entrega': '2022-06-10',
  'nombre_cliente': 'Marcela Cruz',
  'telefono': '93838381005',
  'compra': [{'producto_id': 'jamon', 'cantidad': 6, 'subtotal': 90},
   {'producto_id': 'piña', 'cantidad': 2, 'subtotal': 30},
   {'producto_id': 'zarzamora', 'cantidad': 4, 'subtotal': 80}],
  'total_orden': 200}]

In [86]:
pago_anticipos_por_dia(hoy)

100.0

In [93]:
pago_entregas_por_dia('2022-06-10')

100.0

In [91]:
productos_para_entregas('2022-06-10')


[{'_id': 'jamon', 'cantidad_total': 6},
 {'_id': 'piña', 'cantidad_total': 2},
 {'_id': 'zarzamora', 'cantidad_total': 4}]

In [94]:
pd.DataFrame(productos_para_entregas('2022-06-10'))

,_id,cantidad_total
0,jamon,6
1,piña,2
2,zarzamora,4
